In [44]:
from hlsim import utils, hamiltonian_sim
import numpy as np
from pytket import Circuit
from pytket.passes import DecomposeBoxes
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.extensions.qiskit import tk_to_qiskit
from pytket.extensions.qiskit import AerStateBackend

def build_xxz_trotter_circuit(
    n_qubits: int,
    total_time: float,
    Delta_ZZ: float,
    n_trotter_steps: int = 1,
    order: str = "first",
):
    """
    Constructs a Trotterized circuit for XXZ Hamiltonian time evolution.

    Returns:
    - circ_original : Circuit -> The Trotterized quantum circuit
    - circ_decomposed : Circuit -> Decomposed version showing full gate breakdown
    """
    # 1) Build the XXZ Hamiltonian
    H_xxz = hamiltonian_sim.get_xxz_chain_hamiltonian(n_qubits, Delta_ZZ)

    # 2) Choose first- or second-order Trotterization
    if order.lower() == "first":
        trotter_box = hamiltonian_sim.get_first_order_trotterization(
            hamiltonian=H_xxz,
            n_qubits=n_qubits,
            t_trotterization=total_time,
            n_trotter_steps=n_trotter_steps
        )
    elif order.lower() == "second":
        trotter_box = hamiltonian_sim.get_second_order_trotterization(
            hamiltonian=H_xxz,
            n_qubits=n_qubits,
            t_trotterization=total_time,
            n_trotter_steps=n_trotter_steps
        )
    else:
        raise ValueError("order must be 'first' or 'second'.")

    # 3) Embed the CircBox into a new Circuit
    circ_original = Circuit(n_qubits, name="Trotterized XXZ")
    circ_original.add_circbox(trotter_box, range(n_qubits))

    # 4) Decompose high-level circuit boxes for debugging
    circ_decomposed = circ_original.copy()
    DecomposeBoxes().apply(circ_decomposed)  # Unpacks CircBoxes

    return circ_original, circ_decomposed


##############################
# 2) EXAMPLE USAGE (MAIN)
##############################
if __name__ == "__main__":
    # Define Parameters
    N = 4          # Number of qubits (spin chain size)
    t = 0.1          # Total evolution time
    Delta = 1.0      # Anisotropy parameter
    steps = 2      # Number of Trotter steps
    trotter_order = "first"  # Use "second" for second-order Trotter

    # Generate Circuit
    circ_original, circ_decomposed = build_xxz_trotter_circuit(
        n_qubits=N,
        total_time=t,
        Delta_ZZ=Delta,
        n_trotter_steps=steps,
        order=trotter_order
    )


    sv_backend = AerStateBackend()
    compiled_circuit = sv_backend.get_compiled_circuit(circ_original)


    result = sv_backend.run_circuit(compiled_circuit)
    statevector = result.get_state()


    print("Statevector dimensions: ", statevector.shape)
    print("GHZ Statevector: ", statevector)

    probabilities = np.abs(statevector) ** 2
    print("Probabilities:", probabilities)
    print("Sum of probabilities:", np.sum(probabilities))  # Should be ~1.0


    print("=== Original Trotter Circuit ===")
    draw(circ_decomposed)  # This should now be a proper pytket Circuit

    # # print("\n=== Decomposed Circuit ===")
    # # print(circ_decomposed.get_ascii())  # ASCII-based text visualization

    # # If using Jupyter, visualize with draw()
    # draw(circ_decomposed)

    # # Convert to Qiskit for a detailed visualization
    # qiskit_circ = tk_to_qiskit(circ_decomposed)
    # print("\n=== Qiskit Text Drawer ===")
    # print(qiskit_circ.draw("text"))  # ASCII Qiskit circuit


Statevector dimensions:  (16,)
GHZ Statevector:  [ 9.55336489e-01-2.95520207e-01j  2.37560910e-16-1.43534009e-16j
 -2.77555756e-17-1.11022302e-16j -8.56288071e-17-1.65285915e-15j
  1.57009246e-16+4.31775426e-16j -3.63072937e-16-1.12638690e-17j
 -1.17756934e-16-2.00186788e-15j -2.34374059e-17-1.13931321e-16j
  2.97565780e-17+1.74114415e-16j -1.62753955e-18+2.57338931e-17j
 -3.78353900e-16-2.28615111e-17j  1.77066254e-17+7.39202342e-18j
 -1.08136611e-16-1.69274244e-15j  1.32342170e-18-4.90911583e-18j
 -3.16966648e-19-2.65664949e-20j -5.63261017e-17-7.13328985e-18j]
Probabilities: [1.00000000e+00 7.70371978e-32 1.30963236e-32 2.73927568e-30
 2.11081922e-31 1.31948832e-31 4.02134172e-30 1.35296579e-32
 3.12012836e-32 6.64882139e-34 1.43674322e-31 3.68166593e-34
 2.87707050e-30 2.58508633e-35 1.01173635e-37 3.22351355e-33]
Sum of probabilities: 0.9999999999999998
=== Original Trotter Circuit ===


In [ ]:
from pytket.extensions.qiskit import AerStateBacken

sv_backend = AerStateBackend()

ghz_state = prepare_ghz(n_qubits=4)

result = sv_backend.run_circuit(ghz_state)
statevector = result.get_state()


print("Statevector dimensions: ", statevector.shape)
print("GHZ Statevector: ", statevector)
